# Neo4j Generative AI Workshop Example Application
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-product-examples/genai-workshop/blob/main/genai-example-app-only.ipynb)

 __This notebook contains just the demo application for the LLM content generator and assumes you have already run the main workshop notebook `genai-example-app-only.ipynb`__

In this workshop, you will learn how to use Neo4j Knowledge Graphs to make Large Language Models (LLMs) useful for more real-world use cases.

We walk through an example that uses real-world customer and product data from a fashion, style, and beauty retailer. We show how you can use a knowledge graph to ground an LLM, enabling it to build tailored marketing content personalized to each customer based on their interests and shared purchase histories. We use a pattern called Retrieval-Augmented Generation (RAG) to accomplish this.  Specifically, one that leverages not only vector search but also graph pattern matching and graph machine learning to provide more relevant personalized results to customers.

The workshop walks through the end-to-end process, including the below.  This notebook contains just the final demo application for the LLM content generator.
- Building the knowledge graph
- Vector search & text embedding
- Using graph patterns in Cypher to improve semantic search with context
- Further augmenting semantic search with knowledge graph inference & ML
- Building the LLM chain and demo app for generating content

## Setup

### Some Logics
1. Make a copy of this notebook in Colab by [clicking here](https://colab.research.google.com/github/neo4j-product-examples/genai-workshop/blob/main/genai-workshop.ipynb).
2. Run the pip install below to get the necessary dependencies.  this can take a while. Then run the following cell to import relevant libraries


In [15]:
%%capture
%pip install langchain langchain-openai openai tiktoken python-dotenv gradio neo4j

In [16]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import gradio as gr

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

### Setup Credentials and Environment Variables

There are two things you need here.
1. Start a blank [Neo4j Sandbox](https://sandbox.neo4j.com/). Get your URI and password and plug them in below.  Do not change the Neo4j username.
2. Get your OpenAI API key.  You can use [this one](https://docs.google.com/document/d/19Lqjd0MqRs088KUVnd23ZrVU9G0OAg-53U72VrFwwms/edit) if you do not have one already

To make this easy, you can write the credentials and env variables directly into the below cell.

In [17]:
import os

# Neo4j
NEO4J_URI = 'bolt://34.202.229.218:7687' #change this
NEO4J_PASSWORD = 'terminologies-fire-planet' #change this
NEO4J_USERNAME = 'neo4j'
AURA_DS = False

# AI
LLM = 'gpt-4'

# OpenAI - Required when using OpenAI models
os.environ['OPENAI_API_KEY'] = 'sk-...' #change this

In [26]:
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists('ws-aura.env'):
    load_dotenv('ws-aura.env', override=True)

    # Neo4j
    NEO4J_URI = os.getenv('NEO4J_URI')
    NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
    NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
    AURA_DS = eval(os.getenv('AURA_DS').title())

    # AI
    LLM = os.getenv('LLM')

## LLM For Generating Grounded Content

Let's use an LLM to automatically generate content for targeted marketing campaigns grounded with our knowledge graph using the above tools.
Here is a quick example for generating promotional messages, but you can create all sorts of content with this!

For our first message, let's consider a scenario where a user recently searched for products, but perhaps didn't commit to a purchase yet. We now want to send a message to promote relevant products.

In [27]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
embedding_dimension = 1536

In [28]:
# Import relevant libraries
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

#Instantiate LLM
llm = ChatOpenAI(temperature=0, model_name=LLM, streaming=True)

### Create Knowledge Graph Stores for Retrieval

To ground our content generation, we need to define retrievers to pull information from our knowledge graph.  Let's make two stores:
1. Personalized Search Retriever (`kg_personalized_search`): Based on recent customer searches and purchase history, pull relevant products.
2. Recommendations retriever (`kg_recommendations_app`): Based on our Graph ML, what else can we recommend to them to pair with the relevant products?


In [43]:
# We will use a mock URL for our sources in the metadata

def kg_personalized_search_gen(customer_id):
    return Neo4jVector.from_existing_index(
        embedding=embedding_model,
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        index_name='product_text_embeddings',
        retrieval_query=f"""
        WITH node AS product, score AS searchScore

        OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
        -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {{customerId: '{customer_id}'}})
        WITH count(a) AS purchaseScore, product, searchScore
        RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
            (1.0+purchaseScore)*searchScore AS score,
            {{source: 'https://representative-domain/product/' + product.productCode}} AS metadata
        ORDER BY purchaseScore DESC, searchScore DESC LIMIT 5

    """
    )

# Use the same personalized recommendations as above but with a smaller limit
kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
def kg_recommendations_app(customer_id, k=30):
    res = kg.query("""
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    """, params={'customerId': customer_id, 'k':k})

    return "\n\n".join([d['text'] for d in res])

### Prompt Engineering

Now, let's define our prompts. We will combine two:
1. A system prompt which, in this case, tells the LLM how to generate the message
2. A human prompt that just wraps the customer search(es)/interest(s)

This will allow us to pass the customer interest(s) to the retriever but then also to the LLM for additional context when drafting the message.


In [44]:
general_system_template = '''
You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.
write an email to {customerName}, one of your customers, to promote and summarize products relevant for them given the current season / time of year: {timeOfYear} .
Please only mention the products listed below. Do not come up with or add any new products to the list.
Each product comes with an https `url` field. Make sure to provide that https url with descriptive name text in markdown for each product.

---
# Relevant Products:
{searchProds}

# Customer May Also Be Interested In the following
 (pick items from here that pair with the above products well for the current season / time of year: {timeOfYear}.
 prioritize those higher in the list if possible):
{recProds}
---
'''
general_user_template = "{searchPrompt}"
messages = [
    SystemMessagePromptTemplate.from_template(general_system_template),
    HumanMessagePromptTemplate.from_template(general_user_template),
]
prompt = ChatPromptTemplate.from_messages(messages)

### Create a Chain

Now let's put a chain together that will leverage the retrievers, prompts, and LLM model. This is where Langchain shines, putting RAG together in a simple way.

In addition to the personalized search and recommendations context, we will allow for some other parameters.

1. `timeOfYear`: The time of year as a date, season, month, etc. so the LLM can tailor the language appropriately.
2. `customerName`: Ordinarily, this can be pulled from the DB, but it has been scrubbed to maintain anonymity, so we will provide our own name here.

You can potentially add other creative parameters here to help the LLM write relevant messages.


In [45]:
from langchain.schema.runnable import RunnableLambda

# helper function
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# LLM chain
def chain_gen(customer_id):
    return ({'searchProds': (lambda x:x['searchPrompt']) | kg_personalized_search_gen(customer_id).as_retriever(search_kwargs={"k": 100}) | format_docs,
             'recProds': (lambda x:customer_id) |  RunnableLambda(kg_recommendations_app),
             'customerName': lambda x:x['customerName'],
             'timeOfYear': lambda x:x['timeOfYear'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt
            | llm
            | StrOutputParser())

### Example Runs

In [46]:
# example inputs
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"
search_prompt = 'Oversized Sweaters'

In [47]:
chain = chain_gen(CUSTOMER_ID)

In [48]:
#print(chain.invoke({'searchPrompt':search_prompt, 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

Dear Alex Smith,

I hope this email finds you well. As we are in the heart of winter, I wanted to share some of our cozy and stylish products that are perfect for this season. 

Firstly, our [3p Sneaker Socks](https://representative-domain/product/160442) are short, fine-knit socks designed to be hidden by your shoes with a silicone trim at the back of the heel to keep them in place. They are perfect for keeping your feet warm and comfortable.

Next, our [Hathaway Top](https://representative-domain/product/575141) is a long-sleeved jersey top that is perfect for layering under sweaters or jackets. It's a versatile piece that can be dressed up or down.

For your comfort, we have two bras that you might like. The [Beast wireless push chicago](https://representative-domain/product/751282) is a non-wired lace bra with padded cups for a larger bust and fuller cleavage. The [Orlando C&S Push](https://representative-domain/product/810557) is an underwired bra in lace with removable inserts th

#### Inspecting the Prompt Sent to the LLM
In the above run, the LLM should only be using results from our Neo4j database to populate recommendations. Run the below cell to see the final prompt that was sent to the LLM.

In [49]:
def format_final_prompt(x):
   return f'''=== Prompt to send to LLM ===
   {x.to_string()}
   === End Prompt ===
   '''

def chain_print_prompt(customer_id):
    return ({'searchProds': (lambda x:x['searchPrompt']) | kg_personalized_search_gen(customer_id).as_retriever(search_kwargs={"k": 100}) | format_docs,
             'recProds': (lambda x:customer_id) |  RunnableLambda(kg_recommendations_app),
             'customerName': lambda x:x['customerName'],
             'timeOfYear': lambda x:x['timeOfYear'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt
            | format_final_prompt
            | StrOutputParser())

print( chain_print_prompt(CUSTOMER_ID)\
      .invoke({'searchPrompt':search_prompt, 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

=== Prompt to send to LLM ===
   System: 
You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.
write an email to Alex Smith, one of your customers, to promote and summarize products relevant for them given the current season / time of year: Feb, 2024 .
Please only mention the products listed below. Do not come up with or add any new products to the list.
Each product comes with an https `url` field. Make sure to provide that https url with descriptive name text in markdown for each product.

---
# Relevant Products:
##Product
Name: 3p Sneaker Socks
Type: Socks
Group: Socks & Tights
Garment Type: Socks and Tights
Description: Short, fine-knit socks designed to be hidden by your shoes with a silicone trim at the back of the heel to keep them in place.
url: https://representative-domain/product/160442

##Product
Name: Hathaway
Type: Top
Group: Garment Upper body
Garment Type: Jersey Basic
Description: Long-sleeved jersey top.
url: https://representa

Feel free to experiment and try more!

In [ ]:
#print(chain.invoke({'searchPrompt':"western boots", 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

### Demo App
Now let’s use the above tools to create a demo app with Gradio.  We will need to make a couple more functions, but otherwise easy to fire up from a Notebook!

In [50]:
# Create a means to generate and cache chains...so we can quickly try different customer ids
personalized_search_chain_cache = dict()
def get_chain(customer_id):
    if customer_id in personalized_search_chain_cache:
        return personalized_search_chain_cache[customer_id]
    chain = chain_gen(customer_id)
    personalized_search_chain_cache[customer_id] = chain
    return chain

# create multiple demo examples to try
examples = [
    [
        CUSTOMER_ID,
        'Feb, 2024',
        'Alex Smith',
        'Oversized Sweaters'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'Feb, 2024',
        'Robin Fischer',
        'Oversized Sweaters'
    ],
    [
        '44b0898ecce6cc1268dfdb0f91e053db014b973f67e34ed8ae28211410910693',
        'Feb, 2024',
        'Chris Johnson',
        'Oversized Sweaters'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'Feb, 2024',
        'Robin Fischer',
        'denim jeans'
    ],
]

In [34]:
import gradio as gr

def message_generator(*x):
    chain = get_chain(x[0])
    return chain.invoke({'searchPrompt':x[3], 'customerName':x[2], 'timeOfYear': x[1]})

customer_id = gr.Textbox(value=CUSTOMER_ID, label="Customer ID")
time_of_year = gr.Textbox(value="Feb, 2024", label="Time Of Year")
search_prompt_txt = gr.Textbox(value='Oversized Sweaters', label="Customer Interests(s)")
customer_name = gr.Textbox(value='Alex Smith', label="Customer Name")
message_result = gr.Markdown( label="Message")

demo = gr.Interface(fn=message_generator,
                    inputs=[customer_id, time_of_year, customer_name, search_prompt_txt],
                    outputs=message_result,
                    examples=examples,
                    title="🪄 Message Generator 🥳")
demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /Users/zachblumenfeld/demo/genai-workshop/env/lib/python3.10/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.2
3. Move the file to this location: /Users/zachblumenfeld/demo/genai-workshop/env/lib/python3.10/site-packages/gradio


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None
